In [1]:
import sys
import os
import scrapbook as sb
import pandas as pd
import numpy as np
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import map_at_k, ndcg_at_k, precision_at_k, recall_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams

print("System version: {}".format(sys.version))
print("Pandas version: {}".format(pd.__version__))
print("Tensorflow version: {}".format(tf.__version__))

System version: 3.7.13 (default, Oct 19 2022, 10:19:43) [MSC v.1916 64 bit (AMD64)]
Pandas version: 1.3.5
Tensorflow version: 2.10.0


In [2]:
# top k items to recommend
TOP_K = 5
# Model parameters
EPOCHS = 10
BATCH_SIZE = 1024

SEED = DEFAULT_SEED  # Set None for non-deterministic results

yaml_file = "recommenders/models/deeprec/config/lightgcn.yaml"
user_file = "lightgcn_model/user_embeddings.csv"
item_file = "lightgcn_model/item_embeddings.csv"


train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
vali = pd.read_csv('vali.csv')

data = ImplicitCF(train=train, test=test, seed=SEED)

In [3]:
hparams = prepare_hparams(yaml_file,
                          n_layers=3,
                          batch_size=BATCH_SIZE,
                          epochs=EPOCHS,
                          learning_rate=0.005,
                          eval_epoch=5,
                          top_k=TOP_K,)

In [4]:
model = LightGCN(hparams,data,seed=SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


In [5]:
model.load('lightgcn_model/epoch_100')

In [13]:
with Timer() as train_time:
    model.fit()

print("Took {} seconds for training.".format(train_time.interval))
print(model.each_loss)

Epoch 1 (train)5.3s: train loss = 0.24713 = (mf)0.24498 + (embed)0.00214
Epoch 2 (train)5.2s: train loss = 0.22558 = (mf)0.22316 + (embed)0.00242
Epoch 3 (train)5.2s: train loss = 0.20680 = (mf)0.20410 + (embed)0.00271
Epoch 4 (train)5.5s: train loss = 0.18860 = (mf)0.18560 + (embed)0.00300
Epoch 5 (train)5.1s + (eval)0.5s: train loss = 0.17410 = (mf)0.17081 + (embed)0.00329, recall = 0.04014, ndcg = 0.05998, precision = 0.05081, map = 0.02188
Epoch 6 (train)5.1s: train loss = 0.16050 = (mf)0.15692 + (embed)0.00357
Epoch 7 (train)5.5s: train loss = 0.14894 = (mf)0.14508 + (embed)0.00386
Epoch 8 (train)5.0s: train loss = 0.13803 = (mf)0.13389 + (embed)0.00414
Epoch 9 (train)5.1s: train loss = 0.12958 = (mf)0.12517 + (embed)0.00440
Epoch 10 (train)5.0s + (eval)0.5s: train loss = 0.12104 = (mf)0.11637 + (embed)0.00467, recall = 0.04014, ndcg = 0.06006, precision = 0.05154, map = 0.02161
Took 53.12284120000004 seconds for training.
[0.5914176675753714, 0.4643534089510257, 0.430614427114144

In [12]:
topk_scores = model.recommend_k_items(vali, top_k=3, remove_seen=True,use_id=True)
topk_scores.to_csv("result.csv",index=False)
topk_scores.head(10)

,userID,itemID,prediction
0,0,65,2.190718
1,0,292,1.783050
2,0,250,1.778027
3,1,2340,2.950286
4,1,121,2.916461
5,1,3033,2.717515
6,2,1949,5.456507
7,2,477,5.395557
8,2,565,5.354971
9,3,399,4.418657


In [8]:
eval_map = map_at_k(vali, topk_scores, k=TOP_K)
eval_ndcg = ndcg_at_k(vali, topk_scores, k=TOP_K)
eval_precision = precision_at_k(vali, topk_scores, k=TOP_K)
eval_recall = recall_at_k(vali, topk_scores, k=TOP_K)
print('vali data')
print("MAP:\t%f" % eval_map,
      "NDCG:\t%f" % eval_ndcg,
      "Precision@K:\t%f" % eval_precision,
      "Recall@K:\t%f" % eval_recall, sep='\n')

vali data
MAP:	0.013783
NDCG:	0.033515
Precision@K:	0.021325
Recall@K:	0.017266


TypeError: restore() missing 1 required positional argument: 'save_path'

<bound method get_variable_to_dtype_map of <tensorflow.python.util._pywrap_checkpoint_reader.CheckpointReader object at 0x000001DA4DF101F0>>

AttributeError: module 'tensorflow' has no attribute 'global_variables_initializer'